# Creating an Inference Service using MLFlow and KServe

Welcome to the second part of our tutorial on building a question-answering application over a corpus of private documents using Large Language Models (LLMs). In the previous Notebook, you focused on embedding the documents into a high-dimensional latent space and storing these embeddings in a Vector Store using the Chroma database interface provided by LangChain.

<figure>
  <img src="images/inference-service.jpg" alt="isvc" style="width:100%">
  <figcaption>
    Photo by <a href="https://unsplash.com/@growtika?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Growtika</a> on <a href="https://unsplash.com/photos/GSiEeoHcNTQ?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>
  </figcaption>
</figure>

In this Notebook, you will be taking the next step in this journey. You will use MLFlow to log the Chroma DB files that contain our Vector Store as artifacts of an experiment. After logging the artifacts, you will then create an Inference Service that retrieves these artifacts and uses them to provide context to user queries. For this purpose, you'll be using KServe, a Kubernetes-based platform that provides a serverless framework for serving machine learning models at scale.

It's important to note that since KServe does not support serving a Chroma DB out-of-the-box, you will be using is a custom predictor component. This means that you'll need to create a Docker image first, which can then be deployed as our Inference Service. This process allows for a high degree of customization, enabling us to fine-tune our service to our specific needs. You can find the code, as well as the Dockerfile for the custom predictor inside the `vectorstore` directory of this project. However, you can use the one we have pre-built for you: `gcr.io/mapr-252711/ezua-demos/vectorstore:1.1`.

Once you're ready, this Notebook will guide you through the necessary steps for creating a scalable Vector Store service. First, let's import the libraries you'll need:

In [ ]:
import os
import base64
import logging
import warnings
import subprocess

import mlflow
import requests
import boto3
import s3transfer
import ipywidgets as widgets


warnings.filterwarnings('ignore')

In [ ]:
# Get all loggers
loggers = logging.Logger.manager.loggerDict.values()

# Iterate over all loggers and set their level to ERROR
# as we don't want to polute the output of the code cells
# with debugging messages.
for logger in loggers:
    if isinstance(logger, logging.Logger):
        logger.setLevel(logging.ERROR)

In [ ]:
import base64

def encode_base64(message: str):
    encoded_bytes = base64.b64encode(message.encode('ASCII'))
    return encoded_bytes.decode('ASCII')

# Logging the Vector Store as an Artifact

To kick-off this process, you'll need to provide a few variables:

1. The domain of your EzAF cluster (e.g., `hpe-ezaf`)
1. Your username and password so you can connect to MLFlow, create experiments, and log artifacts.
1. The custom predictor Docker image you built

In [ ]:
# Add heading
heading = widgets.HTML("<h2>MLflow Credentials</h2>")
display(heading)

ezaf_env_input = widgets.Text(description='EZAF Env:')
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
predictor_input = widgets.Text(description='VectorStore image:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

ezaf_env = None
mlflow_username = None
mlflow_password = None
predictor = None

def submit_button_clicked(b):
    global ezaf_env, mlflow_username, mlflow_password, predictor
    ezaf_env = ezaf_env_input.value
    mlflow_username = username_input.value
    mlflow_password = password_input.value
    predictor = predictor_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(ezaf_env_input, username_input, password_input, predictor_input, submit_button, success_message)

In [ ]:
EZAF_ENV = ezaf_env
token_url = f"https://keycloak.{EZAF_ENV}.com/realms/UA/protocol/openid-connect/token"

data = {
    "username" : mlflow_username,
    "password" : mlflow_password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

You'll also need access to the MinIO object store, to log the Chroma DB files. Use the following dialogue to provide your credentials:

In [ ]:
# Add heading
heading = widgets.HTML("<h2>Object Store Credentials</h2>")
display(heading)

# Access Key and Secret Key inputs
access_key_input = widgets.Text(description='Access Key:')
secret_key_input = widgets.Password(description='Secret Key:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

minio_access_key = None
minio_secret_key = None

def submit_button_clicked(b):
    global minio_access_key, minio_secret_key
    minio_access_key = access_key_input.value
    minio_secret_key = secret_key_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(access_key_input, secret_key_input, submit_button, success_message)

Using the following helper function you are ready to create a new experiment (or re-use and existing one) and log the Chroma DB files as an artifact of this experiment. In the end, you'll need to retrieve the URI pointing to the location of this artifact and pass it to the custom predictor component. This way, the custom predictor component will know hot to retrieve the artifact and serve it.

In [ ]:
def get_or_create_experiment(exp_name):
    """Register an experiment in MLFlow.
    
    args:
      exp_name (str): The name of the experiment.
    """
    try:
        user = "dimpo"
        os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = "true"
        os.environ["AWS_ACCESS_KEY_ID"] = minio_access_key
        os.environ["AWS_SECRET_ACCESS_KEY"] = minio_secret_key
        os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"https://home.{EZAF_ENV}.com:31900"
        os.environ['MLFLOW_S3_IGNORE_TLS'] = "true"
        os.environ['MLFLOW_TRACKING_TOKEN'] = token
        mlflow.set_tracking_uri(f"https://mlflow.{EZAF_ENV}.com")
        mlflow.set_experiment(exp_name)
        mlflow.set_tag('mlflow.user', user)
    except Exception as e:
        raise RuntimeError(f"Failed to set the experiment: {e}")

In [ ]:
# Create a new MLFlow experiment or re-use an existing one
get_or_create_experiment('question-answering')

# Log the Chroma DB files as an artifact of the experiment
mlflow.log_artifact(f"{os.getcwd()}/db")

# Retrieve the URI of the artifact
uri = mlflow.get_artifact_uri("db")

# Creating and Submitting the Inference Service

In the final section of this Notebook, you will create and submit an Inference Service using a YAML template and a Python subprocess. The steps in this section include:

1. Creating the YAML Template: You will create a YAML file that defines the specifications of our Inference Service. This includes details like the name of the service, the Docker image to use, and other configuration settings. You will store this YAML into a file that you can explore and submit.
1. Submitting the YAML Template: Once your YAML template is ready, you need to submit it to KServe for deployment. To do this, you will use a Python subprocess to run a shell command that submits our YAML template to KServe.

By the end of this section, you will have a running Inference Service that is ready to receive user queries and provide context for answering them using the Vector Store. This marks the completion of our journey, from transforming unstructured text data into structured vector embeddings, to creating a scalable service that can provide context based on those embeddings.

In [ ]:
isvc = """
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
type: Opaque
data:
  EZAF_ENV: {0}
  MINIO_ACCESS_KEY: {1}
  MINIO_SECRET_KEY: {2}

---
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: vectorstore
spec:
  predictor:
    containers:
    - name: kserve-container
      image: {3}
      imagePullPolicy: Always
      args:
      - --persist-uri
      - {4}
      env:
      - name: TRANSFORMERS_CACHE
        value: /src
      - name: SENTENCE_TRANSFORMERS_HOME
        value: /src
      - name: EZAF_ENV
        valueFrom:
          secretKeyRef:
            key: EZAF_ENV
            name: minio-secret
      - name: MINIO_ACCESS_KEY
        valueFrom:
          secretKeyRef:
            key: MINIO_ACCESS_KEY
            name: minio-secret
      - name: MINIO_SECRET_KEY
        valueFrom:
          secretKeyRef:
            key: MINIO_SECRET_KEY
            name: minio-secret
""".format(encode_base64(ezaf_env),
           encode_base64(minio_access_key),
           encode_base64(minio_secret_key),
           predictor,
           uri)

with open("vectorstore/isvc.yaml", "w") as f:
    f.write(isvc)

In [ ]:
subprocess.run(["kubectl", "apply", "-f", "vectorstore/isvc.yaml"])

# Conclusion and Next Steps

Congratulations! You've successfully navigated through the process of logging the Chroma DB files as artifacts using MLFlow, creating a custom Docker image, and setting up an Inference Service with KServe that retrieves these artifacts to serve your Vector Store. This Inference Service forms the backbone of our question-answering application, enabling us to efficiently answer queries based on the document embeddings we generated previously.

From here, there are two paths you can choose:

- Testing the Vector Store Inference Service: If you'd like to test the Vector Store Inference Service that you've just created, you can proceed to our third (optional) Notebook. This Notebook provides a step-by-step guide on how to invoke the Inference Service and validate its performance.
- Creating the LLM Inference Service: Alternatively, if you're ready to move on to the next stage of the project, you can jump straight to our fourth Notebook. In this Notebook, we'll guide you through the process of creating an Inference Service for the Large Language Model (LLM), which will work in conjunction with the Vector Store Inference Service to provide answers to user queries.